# Dependencies

In [3]:
# !pip install -r requirements.txt

In [4]:
# !pip freeze > requirements.txt

In [5]:
!pip freeze

absl-py==1.4.0
anyio==3.6.2
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
astunparse==1.6.3
attrs==22.2.0
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
cachetools==5.3.0
certifi==2022.12.7
cffi==1.15.1
charset-normalizer==3.1.0
click==8.1.3
comm==0.1.2
contourpy==1.0.7
cycler==0.11.0
debugpy==1.6.6
decorator==5.1.1
defusedxml==0.7.1
executing==1.2.0
fastjsonschema==2.16.3
Flask==2.2.3
flatbuffers==1.12
fonttools==4.39.0
fqdn==1.5.1
gast==0.4.0
google-auth==2.16.2
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.51.3
h5py==3.8.0
idna==3.4
importlib-metadata==6.0.0
importlib-resources==5.12.0
ipykernel==6.21.3
ipython==8.10.0
ipython-genutils==0.2.0
ipywidgets==8.0.4
isoduration==20.11.0
itsdangerous==2.1.2
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.6.3
jupyter_client==8.0.3
jupyter_core==5.2.0
jupyter_server==2.4.0
jupyter_server_termi

# PreProcessing

## Dependencies

In [6]:
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


## Convert data into numpy array

In [7]:
# import pathlib


test_mask = [] 
test_no_mask = []
# I would do this in a fancy os.loop way but I was using google colab and didn't really want to mess with the function too much
train_data_mask_dir="New_Masks_Dataset/Train/Mask"
train_data_no_mask_dir="New_Masks_Dataset/Train/Non_Mask"
val_data_mask_dir = 'New_Masks_Dataset/Validation/Mask'
val_data_no_mask_dir = 'New_Masks_Dataset/Validation/Non_Mask'
test_data_mask_dir="New_Masks_Dataset/Test/Mask"
test_data_no_mask_dir="New_Masks_Dataset/Test/Non_Mask"


In [8]:
# print(train_imagePaths_mask)

In [9]:
# https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
import cv2
# from google.colab.patches import cv2_imshow
import os

IMAGE_HEIGHT  = 100
IMAGE_WIDTH = 100
NUM_CLASSES = 2


input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT)


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        # cv2_imshow(img)
        # Has to be like this because of Python Ambiguity
        if img is not None:
            grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(grayscale, input_shape)
            resized = np.reshape(resized, input_shape[0] * input_shape[1])
            images.append(resized)
    return images


In [10]:
# Train
train_mask = load_images_from_folder(train_data_mask_dir)
train_no_mask = load_images_from_folder(train_data_no_mask_dir)
# Val
val_mask = load_images_from_folder(val_data_mask_dir)
val_no_mask = load_images_from_folder(val_data_no_mask_dir)
# Test
test_mask = load_images_from_folder(test_data_mask_dir)
test_no_mask = (load_images_from_folder(test_data_no_mask_dir))

In [11]:
y_train_mask = [1] * len(train_mask)
y_train_no_mask = [0] * len(train_no_mask)

y_val_mask = [1] * len(val_mask)
y_val_no_mask = [0] * len(val_no_mask)

y_test_mask = [1] * len(test_mask)
y_test_no_mask = [0] * len(test_no_mask)


In [12]:
df_train = pd.DataFrame({'label': y_train_mask,'mask_data': train_mask})

In [13]:
df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35193/1233198709.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)


In [14]:
df_train = df_train.sample(frac = 1)
df_train

,label,mask_data
542,0,"[254, 255, 254, 254, 158, 174, 169, 160, 145, ..."
267,1,"[15, 15, 17, 18, 19, 18, 19, 19, 20, 19, 20, 1..."
207,1,"[24, 26, 27, 27, 28, 29, 29, 33, 32, 32, 31, 3..."
445,0,"[255, 255, 255, 255, 255, 254, 253, 254, 253, ..."
312,0,"[51, 83, 102, 107, 99, 78, 49, 58, 79, 80, 81,..."
...,...,...
400,0,"[124, 113, 127, 126, 126, 125, 124, 125, 125, ..."
416,0,"[255, 255, 255, 255, 255, 255, 254, 253, 252, ..."
504,0,"[254, 254, 231, 143, 145, 149, 149, 154, 151, ..."
4,1,"[103, 107, 108, 110, 106, 104, 103, 100, 96, 1..."


In [15]:
X_train = df_train.iloc[:, 1:] / 255.0
X_train = pd.DataFrame(X_train['mask_data'].tolist())
y_train = df_train.iloc[:, :1]

# .astype('float32')

In [16]:
df_val = pd.DataFrame({'label': y_val_mask,'mask_data': val_mask})
df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)

df_val = df_val.sample(frac = 1)
df_val

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35193/3146285510.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)


,label,mask_data
106,1,"[75, 85, 82, 85, 83, 85, 84, 86, 79, 82, 86, 8..."
96,1,"[111, 106, 107, 107, 110, 110, 105, 106, 103, ..."
18,1,"[118, 132, 140, 152, 157, 165, 170, 172, 176, ..."
257,0,"[170, 187, 169, 128, 127, 124, 123, 136, 137, ..."
126,1,"[17, 16, 19, 15, 16, 19, 21, 18, 24, 25, 28, 3..."
...,...,...
109,1,"[194, 194, 195, 196, 196, 196, 198, 197, 196, ..."
127,1,"[196, 205, 183, 186, 184, 139, 208, 205, 195, ..."
254,0,"[189, 194, 200, 198, 196, 195, 190, 174, 170, ..."
259,0,"[85, 52, 34, 30, 33, 32, 34, 69, 56, 68, 90, 7..."


In [17]:
X_val = df_val.iloc[:, 1:] / 255.0
X_val = pd.DataFrame(X_val['mask_data'].tolist())
y_val = df_val.iloc[:,:1]


In [18]:
df_test = pd.DataFrame({'label': y_test_mask,'mask_data': test_mask})
df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)

df_test = df_test.sample(frac = 1)
df_test


/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35193/4284791088.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)


,label,mask_data
95,0,"[90, 90, 6, 8, 78, 118, 101, 103, 83, 21, 1, 0..."
61,0,"[62, 63, 62, 62, 62, 61, 62, 62, 62, 62, 61, 6..."
60,0,"[208, 198, 208, 209, 223, 222, 216, 212, 196, ..."
48,1,"[236, 236, 236, 236, 236, 236, 236, 236, 236, ..."
90,0,"[219, 219, 219, 219, 219, 218, 218, 217, 218, ..."
...,...,...
36,1,"[229, 230, 230, 230, 231, 231, 229, 227, 224, ..."
39,1,"[196, 196, 196, 197, 197, 197, 197, 193, 192, ..."
58,0,"[140, 144, 143, 94, 59, 68, 60, 41, 86, 52, 58..."
77,0,"[179, 190, 187, 180, 188, 182, 186, 181, 185, ..."


In [19]:
X_test = df_test.iloc[:, 1:] / 255.0
X_test = pd.DataFrame(X_test['mask_data'].tolist())
y_test = df_test.iloc[:, :1]


In [20]:
x_train = X_train.values.reshape(-1,100,100)
x_val = X_val.values.reshape(-1,100,100)
x_test = X_test.values.reshape(-1,100,100)

In [21]:
batch_size = 128
epochs = 15
# input image dimensions
input_shape = (100, 100, 1)

y = y_train #use later for the classifiers
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_val = keras.utils.to_categorical(y_val, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)



In [22]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(599, 100, 100)
(302, 100, 100)
(99, 100, 100)


# Running Model

## KNN (K_Nearest_Neighbor Classifier)

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 30))
param_grid = {'n_neighbors': k_range, 
              'weights': ['uniform', 'distance']}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, verbose = 1)
grid_search.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)
metrics.f1_score(y_test, y_pred, average='weighted')
print("Best Parameters:", grid_search.best_params_)
print("Accuracy Score:", grid_search.best_score_)

Fitting 5 folds for each of 58 candidates, totalling 290 fits
Best Parameters: {'n_neighbors': 22, 'weights': 'distance'}
Accuracy Score: 0.7312605042016808


In [24]:
y_train

array([1., 0.], dtype=float32)

In [25]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score

# lin_classifier = SGDClassifier()
# lin_classifier.fit(X_train, y)
# y_pred = lin_classifier.predict(X_test)

# accuracy_score(y_test, y_pred)


In [26]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Train and evaluate several classifiers
# classifiers = [
#     LogisticRegression(),
#     SVC(),
# ]
# y= y['label']

# for clf in classifiers:
#     clf.fit(X_train, y)
#     y_pred = clf.predict(X_test)
#     print(type(clf).__name__)
#     print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
#     print("Precision: {:.2f}".format(precision_score(y_test, y_pred, average='weighted')))
#     print("Recall: {:.2f}".format(recall_score(y_test, y_pred, average='weighted')))
#     print("F1-score: {:.2f}".format(f1_score(y_test, y_pred, average='weighted')))
#     print()

## CNN (Convolutional neural network)

In [29]:

# The parameters I chose here were also from a project I did earlier, I'll link it here
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation="sigmoid"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 33856)             0         
                                                                 
 dropout_1 (Dropout)         (None, 33856)            

In [31]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
             EarlyStopping(monitor='val_loss', patience=7)]


model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val,y_val))

Epoch 1/15


ValueError: in user code:

    File "/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.8/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/haseebkhan/Desktop/dev/schoolClasses/CISC-4900/lib/python3.8/site-packages/keras/engine/training.py", line 948, in compute_loss
        return se

In [29]:
# model.save('best_model.h5')

In [30]:
# model = keras.models.load_model('./best_model.h5')

In [28]:
model.evaluate(x_test, y_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [32]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.3415547311306
Test accuracy: 0.8585858345031738


In [39]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

print(classification_report(y_true_labels, y_pred_labels))
print(confusion_matrix(y_true_labels, y_pred_labels))

In [40]:
# import cv2
# import tkinter as tk
# from PIL import Image, ImageTk

# class CameraApp:
#     def __init__(self, master):
#         self.master = master
#         master.title("Camera App")

#         # Create a button to take a picture
#         self.button = tk.Button(master, text="Take Picture", command=self.take_picture)
#         self.button.pack()

#         # Create a canvas to display the captured image
#         self.canvas = tk.Canvas(master, width=640, height=480)
#         self.canvas.pack()

#     def take_picture(self):
#         # Open the default camera
#         cap = cv2.VideoCapture(0)

#         # Capture a frame from the camera
#         ret, frame = cap.read()

#         # Save the captured frame as an image
#         self.image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

#         # Release the camera
#         cap.release()

#         # Display the captured image on the canvas
#         self.photo = ImageTk.PhotoImage(self.image)
#         self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

# root = tk.Tk()
# app = CameraApp(root)
# root.mainloop()

In [41]:
# import cv2

# cam = cv2.VideoCapture(0)

# cv2.namedWindow("test")

# img_counter = 0

# while True:
#     ret, frame = cam.read()
#     if not ret:
#         print("failed to grab frame")
#         break
#     cv2.imshow("test", frame)

#     k = cv2.waitKey(1)
#     if k%256 == 27:
#         # ESC pressed
#         print("Escape hit, closing...")
#         break
#     elif k%256 == 32:
#         # SPACE pressed
#         img_name = "opencv_frame_{}.png".format(img_counter)
#         cv2.imwrite(img_name, frame)
#         print("{} written!".format(img_name))
#         img_counter += 1

# cam.release()

# cv2.destroyAllWindows()

In [42]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

In [43]:
img = cv2.imread(os.path.join('opencv_frame_0.png'))
grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(grayscale, input_shape)
resized = np.reshape(resized, input_shape[0] * input_shape[1])


# prediction = model.predict(image)
resized

In [2]:
# https://medium.com/@jinilcs/a-simple-keras-model-on-my-laptop-webcam-dda77521e6a0
import cv2
import numpy as np
from PIL import Image
from keras import models
import pandas as pd

#Load the saved model
model = models.load_model('best_model.h5')
# video = cv2.VideoCapture(0)
input_shape = (100,100)
filename = 'opencv_frame_1.png'

while True:
    _, frame = video.read()

    # frame = cv2.imread(filename)
    #Convert the captured frame into RGB
    im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(im, input_shape)
    resized = np.reshape(resized, input_shape[0] * input_shape[1])
    im_df = pd.DataFrame(resized)
    #Resizing into 100x100 because we trained the model with this image size.
    X = im_df / 255.0
    X = X.values.reshape(-1,100,100)
    #Calling the predict method on model to predict 'me' on the image
    print(model.predict(X))
    prediction = int(model.predict(X)[0][0])

    print(prediction)
#     if prediction is 0, which means I am missing on the image, then show the frame in gray color.
    if prediction == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow("Capturing", frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
            break
# video.release()
# cv2.destroyAllWindows()

(1, 100, 100)
1/1 [==============================] - 0s 77ms/step
[[0.17905957 0.82273626]]
1/1 [==============================] - 0s 33ms/step
0


In [27]:
# https://medium.com/@jinilcs/a-simple-keras-model-on-my-laptop-webcam-dda77521e6a0
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from PIL import Image
from keras import models
import pandas as pd

# Load the saved model
model = load_model('best_model.h5')

# Define a function to make predictions
def predict_mask(image):

    frame = cv2.imread(image)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, input_shape)

    # Convert the image to a numpy array
    image_array = np.array(image)
    
    # Resize the image to the correct dimensions (if necessary)
    image_array = np.resize(image_array, (1, 100, 100))
    
    # Normalize the image data
    image_array = image_array / 255.0
    print(image_array.shape)
    
    # Make a prediction with the model
    prediction = model.predict(image_array)
    print(prediction)
    
    # Return the prediction
    return prediction[0][0] > 0.5  # Returns True if wearing mask, False if not

# Example usage
# image = 'New_Masks_Dataset/Test/Mask/2085.jpg'
# image = 'New_Masks_Dataset/Test/Non_Mask/real_01032.jpg'
# image = 'New_Masks_Dataset/Test/Mask/2114.jpeg'
# image = 'New_Masks_Dataset/Test/Non_Mask/real_01042.jpg'
image = 'opencv_frame_0.png'

prediction = predict_mask(image)
print(prediction)

(1, 100, 100)
1/1 [==============================] - 0s 102ms/step
[[0.0608748  0.94396263]]
False
